In [1]:
import openai
from dotenv import load_dotenv
import os
from IPython.display import display, HTML, JSON, Markdown
from langchain import PromptTemplate
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI
from langchain.chains import LLMChain

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_GPT4o_DEPLOYMENT_NAME = os.getenv("OPENAI_GPT4o_DEPLOYMENT_NAME")


In [2]:
def init_llm(model=OPENAI_GPT4o_DEPLOYMENT_NAME,
             deployment_name=OPENAI_GPT4o_DEPLOYMENT_NAME,
             openai_api_version="2024-02-15-preview",
             temperature=0,
             max_tokens=400
             ):

    llm = AzureChatOpenAI(deployment_name=deployment_name,
                            model=model,
                            openai_api_version=openai_api_version,
                            azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
                            temperature=temperature,
                            max_tokens=max_tokens
                            )
    return llm

###  **Few-shot learning**

This technique could improve model performance by a lot. 
We can use the model to learn from a few examples and then use it to generate text. This is called few-shot learning. We can also use the model to learn from a single example and then use it to generate text. This is called one-shot learning.

In [3]:
template_few_shot = """You are a {profession} answering users questions. 
            More specifically, you are an expert in {expertise}. Answer in a clear and concise manner. Assume that a user is not a subject expert.
            If a question is not clear or not related to {expertise} say: it's not clear or the question is not related to {expertise}.
           
            USER: How do you assess the risk tolerance of a new client?
            ASSISTANT: I begin by having a comprehensive discussion with the client about their financial goals, investments horizon, and comfort level with different levels of risk.
            
            USER: Can you provide an example of a specific risk management strategy you'd recommended to a client in a volatile market situation?
            ASSISTANT: During the market volatility caused by the pandemic, I'd recommended that a client diversify their portfolio further to reduce risk exposure.
            
            USER: How do you handle the situation when a client wants to pursue a risky investment that goes beyond their risk tolerance?
            ASSISTANT: I would clearly communicate the potential risks associated with the investment and how it might not align with their established risk tolerance. 
            
            USER: {question}
            ASSISTANT:
            """

In [4]:
llm = init_llm()
prompt_few_shot = PromptTemplate(template=template_few_shot, input_variables=["profession", "expertise", "question"])
chain = LLMChain(llm=llm, prompt=prompt_few_shot)

res = chain.invoke(input= {"profession":"Financial Trading Consultant",  
                           "expertise":"Risk Management",
                           "question":"How do you use technology or specific financial tools to assist in risk management for your clients?"}
                           )
display(Markdown(res["text"]))

I use various financial tools and technologies to assist in risk management, such as:

1. **Risk Assessment Software**: Tools like Riskalyze help quantify a client's risk tolerance and align their portfolio accordingly.
2. **Portfolio Management Systems**: Software like Morningstar or Bloomberg Terminal provides real-time data and analytics to monitor and manage portfolio risk.
3. **Stress Testing Tools**: These tools simulate different market scenarios to see how a portfolio would perform under various conditions.
4. **Automated Alerts**: Setting up alerts for significant market movements or changes in asset performance to take timely action.
5. **Diversification Algorithms**: Using algorithms to optimize asset allocation and ensure diversification to minimize risk.

These technologies help in making informed decisions and maintaining a balanced risk profile for clients.

In [5]:
# Since we don't save a history of the conversation, the model will fail to answer questions that require context.
res = chain.invoke(input= {"profession":"Financial Trading Consultant",  
                           "expertise":"Risk Management",
                           "question":"Which software do you use?"}
                           )
display(Markdown(res["text"]))

It's not clear or the question is not related to Risk Management.